In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import nltk
from nltk.corpus import stopwords
from collections import Counter

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError


In [ ]:
# Load datasets
fake_politifact = pd.read_csv('politifact_fake.csv')
real_politifact = pd.read_csv('politifact_real.csv')
fake_gossipcop = pd.read_csv('gossipcop_fake.csv')
real_gossipcop = pd.read_csv('gossipcop_real.csv')

# Assign labels: 0 for fake news, 1 for real news
fake_politifact['label'] = 0
real_politifact['label'] = 1
fake_gossipcop['label'] = 0
real_gossipcop['label'] = 1

# Concatenate datasets
data = pd.concat([fake_politifact, real_politifact, fake_gossipcop, real_gossipcop], axis=0, ignore_index=True)


In [ ]:
def clean_text(text):
    # Remove special characters and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Convert text to lowercase and tokenize words
    words = text.lower().split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Join words back into a single string
    cleaned_text = ' '.join(words)
    
    return cleaned_text

data['cleaned_title'] = data['title'].apply(clean_text)


In [ ]:
# Split data into training and testing sets (80% train, 20% test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train_data.shape

In [ ]:
train_data.head()


In [ ]:
test_data.shape

In [ ]:
test_data.head()

In [ ]:
def load_glove_vectors(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

glove_vectors = load_glove_vectors('glove.6B.50d.txt')


In [ ]:
def weighted_average_vectors(docs, vectorizer, word_vectors, dim=50):
    # Calculate the tf-idf weights for the given documents
    tfidf_matrix = vectorizer.fit_transform(docs)

    # Initialize an empty matrix to store the weighted average vectors
    weighted_vectors = np.zeros((len(docs), dim))

    # Iterate through the documents and compute the weighted average vector for each
    for i, doc in enumerate(docs):
        words = doc.split()
        weighted_sum = np.zeros(dim)
        total_weight = 0

        for word in words:
            if word in word_vectors and word in vectorizer.vocabulary_:
                vector = word_vectors[word]
                weight = tfidf_matrix[i, vectorizer.vocabulary_[word]]
                weighted_sum += weight * vector
                total_weight += weight

        if total_weight != 0:
            weighted_vectors[i] = weighted_sum / total_weight

    return weighted_vectors


In [ ]:
vectorizer = TfidfVectorizer(max_features=10000)
X_train = weighted_average_vectors(train_data['cleaned_title'], vectorizer, glove_vectors)
X_test = weighted_average_vectors(test_data['cleaned_title'], vectorizer, glove_vectors)

y_train = train_data['label'].values
y_test = test_data['label'].values


In [ ]:
def create_autoencoder(input_dim, encoding_dim, hidden_layers):
    # Encoder
    encoder_input = Input(shape=(input_dim,))
    encoder_layers = [Dense(hidden_layers[0], activation='relu')(encoder_input)]

    for layer_size in hidden_layers[1:]:
        encoder_layers.append(Dense(layer_size, activation='relu')(encoder_layers[-1]))

    # Encoded representation
    encoded = Dense(encoding_dim, activation='relu')(encoder_layers[-1])

    # Decoder
    decoder_layers = [Dense(hidden_layers[-1], activation='relu')(encoded)]

    for layer_size in reversed(hidden_layers[:-1]):
        decoder_layers.append(Dense(layer_size, activation='relu')(decoder_layers[-1]))

    # Reconstruction
    decoder_output = Dense(input_dim, activation='linear')(decoder_layers[-1])

    # Build the autoencoder model
    autoencoder = Model(encoder_input, decoder_output)
    return autoencoder


In [ ]:
input_dim = X_train.shape[1]
encoding_dim = 32
hidden_layers = [128, 64]

autoencoder = create_autoencoder(input_dim, encoding_dim, hidden_layers)
autoencoder.summary()


In [ ]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
loss = MeanSquaredError()
autoencoder.compile(optimizer=optimizer, loss=loss)

# Train the model
real_headlines = X_train[y_train == 1]  # filter real news headlines
epochs = 300
batch_size = 32
autoencoder.fit(real_headlines, real_headlines, epochs=epochs, batch_size=batch_size, validation_split=0.1)


In [ ]:
# Reconstruct headlines in the test set
X_test_reconstructed = autoencoder.predict(X_test)

# Calculate the reconstruction errors
reconstruction_errors = np.mean((X_test - X_test_reconstructed)**2, axis=1)


In [ ]:
from sklearn.metrics import confusion_matrix

def find_optimal_threshold(errors, y_true):
    min_diff = float('inf')
    optimal_threshold = 0

    for threshold in np.linspace(errors.min(), errors.max(), num=1000):
        y_pred = (errors > threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        fpr = fp / (fp + tn)  # False Positive Rate
        fnr = fn / (fn + tp)  # False Negative Rate

        diff = abs(fpr - fnr)

        if diff < min_diff:
            min_diff = diff
            optimal_threshold = threshold

    return optimal_threshold

threshold = find_optimal_threshold(reconstruction_errors, y_test)


In [ ]:
y_pred = (reconstruction_errors > threshold).astype(int)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Autoencoder Classifier Metrics:\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")



In [ ]:
from sklearn.linear_model import LogisticRegression

# Train the logistic regression model with an increased number of iterations
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Make predictions
y_pred_log_reg = log_reg.predict(X_test)

# Calculate metrics
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
precision_log_reg = precision_score(y_test, y_pred_log_reg)
recall_log_reg = recall_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg)

print(f"Logistic Regression Metrics:\nAccuracy: {accuracy_log_reg}\nPrecision: {precision_log_reg}\nRecall: {recall_log_reg}\nF1 Score: {f1_log_reg}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train the random forest model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf.predict(X_test)

# Calculate metrics
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f"Random Forest Metrics:\nAccuracy: {accuracy_rf}\nPrecision: {precision_rf}\nRecall: {recall_rf}\nF1 Score: {f1_rf}")
